In [14]:
from PyPDF2 import PdfReader

# Extract text from PDF
def extract_text_from_pdf(file_path):
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

# Example usage
file_path = "Swiggy.pdf"  # Replace with the path to your PDF
pdf_text = extract_text_from_pdf(file_path)
print("Extracted Text Preview:")
print(pdf_text[:500])  # Preview the first 500 characters

Extracted Text Preview:
Annual Report
2023–2024 
  
  
INDEX  
S. NO.  Document Name  Page No. 
1.  Corporate Information  1-2 
2.  Board ’s Report  
 3-29 
3.  Independent Auditor ’s Report on 
Standalone Financial Statement  
 30-42 
4.  Standalone Financial Statement  43-98 
5.  Independent Auditor ’s Report on 
Consolidated Financial Statement  99-108 
6.  Consolidated Financial Statement  109-167 Corporate information
ANAND KRIPALU
Chairman and 
Independent Director
MD & Global CEO –EPL 
Limited, Ex –Diageo, 
Cadb


In [15]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer

# Extractive summarization
def extractive_summary(text, num_sentences=5):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = LsaSummarizer()
    summary = summarizer(parser.document, num_sentences)
    return " ".join([str(sentence) for sentence in summary])

# Generate summary
extractive_summary_text = extractive_summary(pdf_text, num_sentences=5)
print("Extractive Summary:")
print(extractive_summary_text)

Extractive Summary:
Nominee Director (Non -Executive) Prosus Services B.V. Board of Directors 1Key Managerial Personnel Rahul Bothra -Chief Financial Officer M. Sridhar -Company Secretary and Compliance OfficerRegistrar & Transfer Agents Link Intime India Private Limited, C-101, 1st Floor, 247 Park L.B.S. Marg, Vikhroli West, Mumbai 400 083, Maharashtra, India Tel: +91 810 811 6767 Website: www.linkintime.co.in E-mail: rnt.helpdesk@linkintime.co.in Statutory Auditors B S R & Co. LLP, Chartered Accountants Registered And Corporate Office No. Our sharp focus on innovation coupled with strong execution yielded yet another milestone - we became one of the very few global food delivery platforms to achieve EBITDA profitability in less than 9 years since inception. (xiii) 1,23,411 (One Lakh Twenty -Three Thousand Four Hundred and Eleven) Series J2 Compulsorily Convertible Preference Shares of face value INR 10 (Indian Rupees ten only) each. Employees and Third Parties can also make a Protect

In [16]:
from transformers import pipeline

# Load a pre-trained summarization model
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Abstractive summarization
def abstractive_summary(text, max_length=200, min_length=50):
    return summarizer(text, max_length=max_length, min_length=min_length, do_sample=False)[0]['summary_text']

# Generate summary (split text into smaller chunks if it's very long)
chunk_size = 1024  # Hugging Face models have a max token limit
chunks = [pdf_text[i:i+chunk_size] for i in range(0, len(pdf_text), chunk_size)]

# Summarize each chunk and combine
abstractive_summary_text = " ".join([abstractive_summary(chunk) for chunk in chunks])
print("Abstractive Summary:")
print(abstractive_summary_text)

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


KeyboardInterrupt: 

In [17]:
import torch
print("GPU Available:", torch.cuda.is_available())

GPU Available: False


In [18]:
import torch
print("MPS Available:", torch.backends.mps.is_available())
print("MPS Built:", torch.backends.mps.is_built())

MPS Available: True
MPS Built: True


In [19]:
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print(f"Using device: {device}")

Using device: mps


In [20]:
from transformers import pipeline
import torch

device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=0 if device.type == "mps" else -1)

# Example usage
text = "Your financial report text goes here. This text will be summarized using the MPS backend."
summary = summarizer(text, max_length=150, min_length=50, do_sample=False)
print("Summary:", summary[0]['summary_text'])

Your max_length is set to 150, but your input_length is only 20. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=10)
/Users/vanshdeepmadan/opt/anaconda3/lib/python3.8/site-packages/transformers/pytorch_utils.py:325: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_elements = torch.tensor(test_elements)


Summary: 


In [21]:
from transformers import pipeline
import torch
from PyPDF2 import PdfReader

# Set the device to MPS (Apple GPU) if available, otherwise fallback to CPU
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print(f"Using device: {device}")

# Load the pre-trained summarization model with MPS support
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=0 if device.type == "mps" else -1)

# Function to extract text from a PDF file
def extract_text_from_pdf(file_path):
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

# Abstractive summarization function
def abstractive_summary(text, max_length=200, min_length=50):
    return summarizer(text, max_length=max_length, min_length=min_length, do_sample=False)[0]['summary_text']

# Chunk and summarize the text
def summarize_large_pdf(file_path, chunk_size=1024, max_length=200, min_length=50):
    pdf_text = extract_text_from_pdf(file_path)
    chunks = [pdf_text[i:i+chunk_size] for i in range(0, len(pdf_text), chunk_size)]
    summaries = [abstractive_summary(chunk, max_length=max_length, min_length=min_length) for chunk in chunks]
    return " ".join(summaries)

# Example usage
file_path = "Swiggy.pdf"  # Replace with the path to your PDF
final_summary = summarize_large_pdf(file_path)
print("Final Summary:")
print(final_summary)

Using device: mps


Your max_length is set to 200, but your input_length is only 102. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=51)


Final Summary:
                                                   


In [22]:
from transformers import pipeline
import torch
from PyPDF2 import PdfReader

# Set the device to MPS (Apple GPU) if available, otherwise fallback to CPU
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print(f"Using device: {device}")

# Load the pre-trained summarization model with MPS support
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=0 if device.type == "mps" else -1)

# Function to extract text from a PDF file
def extract_text_from_pdf(file_path):
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

# Dynamic summarization with adjusted max_length
def abstractive_summary(text):
    input_length = len(text.split())  # Calculate input length in tokens (approximation)
    max_length = min(200, max(30, int(input_length * 0.5)))  # Set max_length to 50% of input_length, with bounds
    return summarizer(text, max_length=max_length, min_length=int(max_length * 0.5), do_sample=False)[0]['summary_text']

# Chunk and summarize the text
def summarize_large_pdf(file_path, chunk_size=1024):
    pdf_text = extract_text_from_pdf(file_path)
    chunks = [pdf_text[i:i+chunk_size] for i in range(0, len(pdf_text), chunk_size)]
    summaries = [abstractive_summary(chunk) for chunk in chunks]
    return " ".join(summaries)

# Example usage
file_path = "Swiggy.pdf"  # Replace with the path to your PDF
final_summary = summarize_large_pdf(file_path)
print("Final Summary:")
print(final_summary)

Using device: mps
Final Summary:
                                                   


In [3]:
from transformers import pipeline
import torch
from PyPDF2 import PdfReader

# Set the device to MPS (Apple GPU) if available, otherwise fallback to CPU
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print(f"Using device: {device}")

# Load the pre-trained summarization model with MPS support
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=0 if device.type == "mps" else -1)

# Function to extract text from a PDF file
def extract_text_from_pdf(file_path):
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

# Dynamic summarization with adjusted max_length
def abstractive_summary(text):
    input_length = len(text.split())  # Calculate input length in tokens (approximation)
    max_length = min(200, max(30, int(input_length * 0.5)))  # Set max_length to 50% of input_length, with bounds
    return summarizer(text, max_length=max_length, min_length=int(max_length * 0.5), do_sample=False)[0]['summary_text']

# Chunk and summarize the text
def summarize_large_pdf(file_path, chunk_size=1024):
    pdf_text = extract_text_from_pdf(file_path)
    chunks = [pdf_text[i:i+chunk_size] for i in range(0, len(pdf_text), chunk_size)]
    summaries = [abstractive_summary(chunk) for chunk in chunks]
    return " ".join(summaries)

# Example usage
file_path = "zomato.pdf"  # Replace with the path to your PDF
final_summary = summarize_large_pdf(file_path)
print("Final Summary:")
print(final_summary)

Using device: mps
Final Summary:
                                                                        
